In [19]:
import re
import difflib
import pandas as pd
from datetime import datetime
from typing import List, Callable, Dict

In [20]:
# Load the customers dataset
customers_df = pd.read_csv(
    "../data/01_raw/olist_customers_dataset.csv",
    converters={
        'customer_zip_code_prefix': str # Convert ZIP codes to strings
    }
)
customers_df

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,03937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,06764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [21]:
# Load the geolocation dataset
geolocation_df = pd.read_csv(
    "../data/01_raw/olist_geolocation_dataset.csv",
    converters={
        'geolocation_zip_code_prefix': str # Convert ZIP codes to strings
    }
)
geolocation_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [22]:
# Load the orders dataset
orders_df = pd.read_csv("../data/01_raw/olist_orders_dataset.csv")
orders_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [23]:
# Load the products dataset
products_df = pd.read_csv("../data/01_raw/olist_products_dataset.csv")
products_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [24]:
def drop_rows_conditional(df: pd.DataFrame, conditions: dict) -> pd.DataFrame:
    """
    Removes rows from the DataFrame that do not match the specified conditions.

    Args:
        df (pd.DataFrame): The input DataFrame.
        conditions (dict): A dictionary where keys are column names and values are the conditions.

    Returns:
        pd.DataFrame: The DataFrame with rows matching the conditions removed.
    """
    try:
        # Initialize the mask for rows to keep
        mask = pd.Series(True, index=df.index)
        
        for column, condition in conditions.items():
            # Apply the condition and remove rows which do not meet the condition
            mask &= df[column].apply(condition)
        
        # Return the filtered DataFrame
        return df[mask]
    
    except Exception as e:
        # Showcase the error
        print(f"An unexpected error occurred: {e}")
        return df


In [25]:
conditions = {
    'geolocation_lat': lambda x: -33.742 < x < 5.272,
    'geolocation_lng': lambda x: -73.992 < x < -34.792
}

In [26]:
drop_rows_conditional(geolocation_df, conditions)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [27]:
def drop_rows_missing_values(df: pd.DataFrame, subset: List[str], how: str) -> pd.DataFrame:
    """
    Removes rows from the DataFrame that have missing values.

    Args:
        df (pd.DataFrame): The input DataFrame.
        subset (List[str]): The columns to search for missing values.
        how (str): Determines whether to drop rows with any missing or all missing values.

    Returns:
        pd.DataFrame: The DataFrame with the missing values removed from the specified columns.
    """
    try:
        # Ensure the 'how' argument is valid
        if how not in ['any', 'all']:
            raise ValueError("The 'how' parameter must be either 'any' or 'all'.")
        
        # Check if columns exist in the DataFrame
        missing_columns = [col for col in subset if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")

        # Drop rows where the specified columns/fields have null entries
        return df.dropna(subset=subset, how=how)
    
    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df

In [28]:
drop_rows_missing_values(products_df, subset=list(products_df.columns.drop('product_id')), how='all')

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [29]:
products_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


In [30]:
def drop_columns(df: pd.DataFrame, columns_to_drop: List[str]) -> pd.DataFrame:
    """
    Drops the specified columns in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        columns_to_drop (List[str]): List of the specified columns to be removed from the Dataframe.

    Returns:
        pd.DataFrame: The DataFrame with the specified columns removed.
    """
    try:
        # Check if all specified columns exist in the DataFrame
        missing_columns = [col for col in columns_to_drop if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")
        
        # Drop the specified columns from the DataFrame
        return df.drop(columns=columns_to_drop)
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return df 
    
drop_columns(geolocation_df, columns_to_drop=['geolocation_lng', 'geolocation_lat'])

,geolocation_zip_code_prefix,geolocation_city,geolocation_state
0,01037,sao paulo,SP
1,01046,sao paulo,SP
2,01046,sao paulo,SP
3,01041,sao paulo,SP
4,01035,sao paulo,SP
...,...,...,...
1000158,99950,tapejara,RS
1000159,99900,getulio vargas,RS
1000160,99950,tapejara,RS
1000161,99980,david canabarro,RS


In [31]:
geolocation_df

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
1000158,99950,-28.068639,-52.010705,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS
1000161,99980,-28.388932,-51.846871,david canabarro,RS


In [32]:
def aggregate_by_column(df: pd.DataFrame, column: str, agg: dict) -> pd.DataFrame:
    """
    Aggregates the dataset by the specified column according to the specified aggregation functions.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The column to group by.
        agg (str): The dictionary to aggregate the other columns by.

    Returns:
        pd.DataFrame: The aggregated DataFrame.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Aggregate the input DataFrame by the specified column
        aggregated_df = df.groupby(column).agg(agg)

        # Return the aggregated Dataframe
        return aggregated_df
    
    except ValueError as ve:
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        # Show the error
        print(f"An unexpected error occurred: {e}")
        return df

agg = {
    'geolocation_lat': 'median',
    'geolocation_lng': 'median',
    'geolocation_city': lambda x: x.mode().iloc[0],
    'geolocation_state': lambda x: x.mode().iloc[0]
}


aggregate_by_column(geolocation_df, column='geolocation_zip_code_prefix', agg=agg)

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
01001,-23.550381,-46.634027,sao paulo,SP
01002,-23.548551,-46.635072,sao paulo,SP
01003,-23.548977,-46.635313,sao paulo,SP
01004,-23.549535,-46.634771,sao paulo,SP
01005,-23.549612,-46.636532,sao paulo,SP
...,...,...,...,...
99960,-27.953797,-52.029641,charrua,RS
99965,-28.179542,-52.035551,agua santa,RS
99970,-28.343273,-51.873734,ciriaco,RS


In [33]:
    
def replace_diacritics(text: str) -> str:
    """
    Replaces diacritics with standard alphabets.

    Args:
        text (str): The input string which may contain diacritic.

    Returns:
        str: Outputs a string where the diacritics is replaced with standard alphabet letters.
    """
    # Initialise the regular expressions for diacritics conversion
    replacements = {
        r'[ãââàáä]': 'a',
        r'[íîì]': 'i',
        r'[úûùü]': 'u',
        r'[éêèë]': 'e',
        r'[óõôòö]': 'o',
        r'[ç]': 'c'
    }

    # Checks if the text is a string
    if isinstance(text, str):
        for pattern, replacement in replacements.items():
            text = re.sub(pattern, replacement, text)
        return text
    else:
        raise ValueError("Text must be a string")

def identify_closest_text(text: str, text_list: List[str]) -> str:
    """
    Identify closest text from the list of text.

    Args:
        text (str): The input string to be compared.
        text_list (List[str]): The list of text to be compared with the text to identify the closest one.

    Returns:
        str: The closest text in the list.
    """
    try:
        # Identify the closest text
        closest_text = difflib.get_close_matches(text, text_list)[0]
        return closest_text
    
    except Exception as e:
        # Show error
        print(f"An unexpected error occurred: {e}")

def standardize_cities(df: pd.DataFrame, city_column: str, city_list: List[str]) -> pd.DataFrame:
    """
    Standardizes the names of the cities in the city column of the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        city_column (str): The column containing the city names.
        city_list (List[str]): The list containing the preprocessed true names of all the cities in Brazil without encoding errors or diacritics e.g. sao paulo.

    Returns:
        pd.DataFrame: The DataFrame with standardised names for the cities.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if city_column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Replaces diacritics with plain alphabets
        df[city_column] = df[city_column].apply(replace_diacritics)

        # Identifies the closest city match
        df[city_column] = df[city_column].apply(lambda x: identify_closest_text(x, city_list))
        
        return df
    
    except ValueError as ve:
        # Show error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show error
        print(f"An unexpected error occurred: {e}")
        return df


In [34]:
# Determine the list of citites
customers_df['customer_city'].value_counts()

customer_city
sao paulo                15540
rio de janeiro            6882
belo horizonte            2773
brasilia                  2131
curitiba                  1521
                         ...  
marques de souza             1
general maynard              1
cerrito                      1
mocajuba                     1
governador mangabeira        1
Name: count, Length: 4119, dtype: int64

In [35]:

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [36]:
    
# standardize_cities(geolocation_df, city_column='geolocation_city', city_list=''):



In [37]:

def impute_entries(df: pd.DataFrame, imputation_parameters: dict) -> pd.DataFrame:
    """
    Imputes entries in the DataFrame using MICE.
    
    Args:
        df (pd.DataFrame): The dataset with missing entries.
        imputation_parameters (dict): A dictionary that contains the parameters for imputing the missing age entries
            - significant_numerical_columns (List[str]): The numerical columns which are significant for the MICE imputer to fit by.
            - significant_categorical_columns (List[str]): The categorical columns which are significant for the MICE imputer to fit by.
            - max_iter (int, optional): The maximum number of imputation iterations. Default is 10.
            - random_state (int, optional): The seed used by the MICE imputer. Default is 0.
    
    Returns:
        pd.DataFrame: The dataset with imputed entries.
    """
    try:
        # Identify the numerical and categorical columns from the significant columns
        numerical_cols = imputation_parameters.get('significant_numerical_columns', [])
        categorical_cols = imputation_parameters.get('significant_categorical_columns', [])

        # Get the max iteration and random state for the MICE imputer
        max_iter = imputation_parameters.get('max_iter', 10)
        random_state = imputation_parameters.get('random_state', 42)

        # Define a list containing all the significant columns above
        significant_cols = numerical_cols + categorical_cols

        # Check if these columns exist in the DataFrame
        missing_columns = [col for col in significant_cols if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Columns not found in the DataFrame: {', '.join(missing_columns)}")

        # Identify the columns which are not used in the imputation
        columns_not_used = [col for col in df.columns if col not in significant_cols]

        # Instantiate the IterativeImputer (MICE imputer), OneHotEncoder and StandardScaler
        mice_imputer = IterativeImputer(max_iter=max_iter, 
                                        random_state=random_state,
                                        min_value=0.17,
                                        max_value=80,
                                        initial_strategy='median')
        
        ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        scaler = StandardScaler()

        # Obtain the subset of the dataset
        imputing_df = df[significant_cols]

        # Scale the numerical fields with standard scaling
        if numerical_cols:
            scaled_numerical_data = scaler.fit_transform(imputing_df[numerical_cols])
            scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_cols, index=imputing_df.index)
        else:
            scaled_numerical_df = pd.DataFrame()

        # One hot encode the categorical fields
        if categorical_cols:
            encoded_categorical_data = ohe_encoder.fit_transform(imputing_df[categorical_cols])
            encoded_categorical_df = pd.DataFrame(
                encoded_categorical_data,
                columns=ohe_encoder.get_feature_names_out(categorical_cols),
                index=imputing_df.index
            )
        else:
            encoded_categorical_df = pd.DataFrame()

        # Combine the processed numerical and categorical data
        processed_df = pd.concat([scaled_numerical_df, encoded_categorical_df], axis=1)

        # Apply MICE imputation and convert the data to a dataframe
        imputed_data = mice_imputer.fit_transform(processed_df)
        imputed_df = pd.DataFrame(imputed_data, columns=processed_df.columns, index=processed_df.index)

        # Restore the numerical scaling to the original
        if numerical_cols:
            restored_numerical_data = scaler.inverse_transform(imputed_df[numerical_cols])
            restored_numerical_df = pd.DataFrame(
                restored_numerical_data, columns=numerical_cols, index=imputed_df.index
            )
        else:
            restored_numerical_df = pd.DataFrame()

        # Restore categorical columns to their original categories
        if categorical_cols:
            restored_categorical_data = ohe_encoder.inverse_transform(imputed_df[ohe_encoder.get_feature_names_out(categorical_cols)])
            restored_categorical_df = pd.DataFrame(
                restored_categorical_data, columns=categorical_cols, index=imputed_df.index
            )
        else:
            restored_categorical_df = pd.DataFrame()

        # Combine the restored numerical and categorical columns
        restored_df = pd.concat([restored_numerical_df, restored_categorical_df], axis=1)

        # Combine the restored dataframe with the other columns that were not used
        if columns_not_used:
            restored_df[columns_not_used] = df[columns_not_used]

        return restored_df
    
    except ValueError as ve:
        # Show the error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show the error
        print(f"An unexpected error occurred: {e}")
        return df

In [38]:
imputation_parameters = {
    'significant_numerical_columns': list(products_df.columns.drop(['product_id', 'product_category_name'])),
    'significant_categorical_columns': [],
    'max_iter': 100,
    'random_state': 42
}

In [47]:
list(products_df.columns.drop(['product_id', 'product_category_name']))

['product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [39]:
dataset = impute_entries(products_df, imputation_parameters)
dataset[dataset.isna().any(axis=1)]

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_id,product_category_name
105,50.218698,879.463204,2.484232,650.0,17.0,14.0,12.0,a41e356c76fab66334f36de622ecbd3a,NaN
128,50.218698,879.463204,2.484232,300.0,16.0,7.0,20.0,d8dee61c2034d6d075997acef1870e9b,NaN
145,50.218698,879.463204,2.484232,200.0,20.0,20.0,20.0,56139431d72cd51f19eb9f7dae4d1617,NaN
154,50.218698,887.809180,2.484232,18500.0,41.0,30.0,41.0,46b48281eb6d663ced748f324108c733,NaN
197,50.218698,879.463204,2.484232,300.0,35.0,7.0,12.0,5fb61f482620cb672f5e586bb132eae9,NaN
...,...,...,...,...,...,...,...,...,...
32515,50.878971,879.463204,2.484232,1800.0,30.0,20.0,70.0,b0a0c5dd78e644373b199380612c350a,NaN
32589,50.218698,879.463204,2.484232,800.0,30.0,10.0,23.0,10dbe0fbaa2c505123c17fdc34a63c56,NaN
32616,50.218698,879.463204,2.484232,200.0,21.0,8.0,16.0,bd2ada37b58ae94cc838b9c0569fecd8,NaN
32772,50.230558,879.463204,2.484232,1300.0,45.0,16.0,45.0,fa51e914046aab32764c41356b9d4ea4,NaN


In [40]:
products_df[products_df.isna().any(axis=1)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,NaN,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,NaN,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


In [41]:
dataset.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32951.000000,32951.00000,32951.000000,32951.000000,32951.000000,32951.000000,32951.000000
mean,48.509987,773.58436,2.194489,2276.517662,30.815252,16.937802,23.196853
std,10.153354,629.40346,1.721082,4281.912701,16.913960,13.637152,12.078691
min,5.000000,4.00000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,344.00000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,604.00000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,961.24668,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.00000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [42]:
products_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


In [43]:
def simple_impute_entries(df: pd.DataFrame, column: str, value: str) -> pd.DataFrame:
    """
    Imputes missing entries in the DataFrame using a specified value.
    
    Args:
        df (pd.DataFrame): The input dataset with missing entries.
        column (str): The column to impute.
        value (str): The value to impute into the empty cell.
    
    Returns:
        pd.DataFrame: The dataset with imputed entries.
    """
    try:
        # Check if the column exist in the DataFrame
        is_column_missing = True if column not in df.columns else False
        if is_column_missing:
            raise ValueError(f"Column not found in the DataFrame")
        
        # Impute the empty cells in the column with the specified value
        df[column] = df[column].fillna(value)
        return df
        
    except ValueError as ve:
        # Show error
        print(f"ValueError: {ve}")
        return df
    
    except Exception as e:
        # Show error
        print(f"An unexpected error occurred: {e}")
        return df

In [44]:
dataset = simple_impute_entries(products_df, column='product_category_name', value='Missing')
dataset[dataset.isna().any(axis=1)]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,Missing,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,Missing,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,Missing,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,Missing,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,Missing,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,Missing,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,Missing,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,Missing,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,Missing,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


In [45]:
def date_difference(row: pd.Series, params: dict) -> int:
    """
    Calculate the difference in days between two dates.

    Parameters:
        row (pd.Series): The row from the dataset.
        mapping (dict): A dictionary containing:
            - 'date1': Column name for the first date.
            - 'date2': Column name for the second date.
            - 'date1_format': Format of the first date.
            - 'date2_format': Format of the second date.

    Returns:
        int: The difference in days between the two dates.
    """
    try:
        # Identify the 2 dates
        date1 = row[params['date1']]
        date2 = row[params['date2']]

        # Check if there are any missing dates
        if pd.isna(date1) or pd.isna(date2):
            return None # Returns None when its not delivered yet
        
        # Parse the dates
        d1 = datetime.strptime(date1, params['date1_format'])
        d2 = datetime.strptime(date2, params['date2_format'])

        # Calculate the difference
        day_difference = (d2 - d1).days
        return day_difference
    
    except ValueError as ve:
        raise ValueError(f"Invalid date or format: {ve}")
    
def feature_engineering(df: pd.DataFrame, new_feature: str, function: Callable, mapping: Dict[str, str]) -> pd.DataFrame:
    """
    Create a new feature based on the features in the row.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        new_feature (str): The column name of the new feature to be created.
        function (Callable): The function to be applied in order to create the feature.
        mapping (List[str]): Mapping column names to the parameter names used by the function. Provides other arguments for the function.

    Returns:
        pd.DataFrame: The DataFrame with new features.
    """
    try:
        # Check if the function is valid
        if not callable(function):
            raise ValueError(f"Function is invalid or not callable")
        
        # Initialise the feature engineered DataFrame
        feature_engineered_df = df.copy()
        
        # Apply the function to each row and create the new feature
        feature_engineered_df[new_feature] = df.apply(lambda row: function(row, mapping), axis=1)
        return feature_engineered_df

    except ValueError as ve:
        # Show error
        print(f"ValueError: {ve}")
        return df

    except Exception as e:
        # Show the error
        print(f"An error occurred during feature engineering: {e}")
        return df

In [46]:
mapping = {
    'date1': "order_delivered_customer_date",
    'date2': "order_estimated_delivery_date",
    'date1_format': "%Y-%m-%d %H:%M:%S",
    'date2_format': "%Y-%m-%d %H:%M:%S",
}

feature_engineering(orders_df, new_feature='estimated_actual_delivery_offset', function=date_difference, mapping=mapping).sort_values('estimated_actual_delivery_offset')

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,estimated_actual_delivery_offset
55619,1b3190b2dfa9d789e1f14c05b647a14a,d306426abe5fca15e54b645e4462dc7b,delivered,2018-02-23 14:57:35,2018-02-23 15:16:14,2018-02-26 18:49:07,2018-09-19 23:24:07,2018-03-15 00:00:00,-189.0
19590,ca07593549f1816d26a572e06dc1eab6,75683a92331068e2d281b11a7866ba44,delivered,2017-02-21 23:31:27,2017-02-23 02:35:15,2017-03-08 13:47:46,2017-09-19 14:36:39,2017-03-22 00:00:00,-182.0
11399,47b40429ed8cce3aee9199792275433f,cb2caaaead400c97350c37a3fc536867,delivered,2018-01-03 09:44:01,2018-01-03 10:31:15,2018-02-06 01:48:28,2018-07-13 20:51:31,2018-01-19 00:00:00,-176.0
81401,2fe324febf907e3ea3f2aa9650869fa5,65b14237885b3972ebec28c0f7dd2220,delivered,2017-03-13 20:17:10,2017-03-13 20:17:10,2017-03-17 07:23:53,2017-09-19 17:00:07,2017-04-05 00:00:00,-168.0
89130,285ab9426d6982034523a855f55a885e,9cf2c3fa2632cee748e1a59ca9d09b21,delivered,2017-03-08 22:47:40,2017-03-08 22:47:40,2017-03-09 08:53:20,2017-09-19 14:00:04,2017-04-06 00:00:00,-167.0
...,...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00,NaN
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaN,NaN,2018-02-06 00:00:00,NaN
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00,NaN
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00,NaN
